In [183]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, avg, round, when
from pyspark.sql.types import FloatType

In [184]:
spark = SparkSession.builder \
    .appName("Fuel Prices in Brazil Analysis") \
    .getOrCreate()

In [185]:
df_gasoline_ethanol = spark.read.format("parquet").load("Gasoline and Ethanol Prices.parquet")
df_gasoline_ethanol.show()

+--------------+--------------+---------------+--------------------+-------------------+--------------------+----------+--------------------+-------------------+---------+------------------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|      Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|         Complemento|             Bairro|      Cep|           Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------------+--------------------+-------------------+--------------------+----------+--------------------+-------------------+---------+------------------+--------------+--------------+---------------+-----------------+--------+
|            SE|            SP|       SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|       306|                null|     JARDIM ZULMIRA|18061-000|          GASOLINA|    24/03/2025|   

In [186]:
df_diesel_cng = spark.read.format("parquet").load("Diesel and CNG Prices.parquet")
df_diesel_cng.show()

+--------------+--------------+-------------------+--------------------+-------------------+--------------------+----------+--------------------+------------------+---------+----------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|          Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|         Complemento|            Bairro|      Cep|   Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+-------------------+--------------------+-------------------+--------------------+----------+--------------------+------------------+---------+----------+--------------+--------------+---------------+-----------------+--------+
|             N|            AC|    CRUZEIRO DO SUL|AMAZONIA COMERCIO...| 01.492.748/0003-83|  AVENIDA COPACABANA|       440|                null|        COPACABANA|69980-000|    DIESEL|    27/03/2025|          8,17|        

In [187]:
df_lpg = spark.read.format("parquet").load("LPG Prices.parquet")
df_lpg.show()

+--------------+--------------+-------------+--------------------+-------------------+--------------------+----------+--------------------+--------------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|    Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|         Complemento|              Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+-------------+--------------------+-------------------+--------------------+----------+--------------------+--------------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|            CO|            MS| CAMPO GRANDE|COMPANHIA ULTRAGA...| 61.602.199/0024-09|RUA AMARO CASTRO ...|      1852|                null|VILA NOVA CAMPO G...|79106-361|    GLP|    24/03/2025|        110,00|

In [188]:
df_fuels = df_gasoline_ethanol.union(df_diesel_cng).union(df_lpg)
df_fuels.show()

+--------------+--------------+---------------+--------------------+-------------------+--------------------+----------+--------------------+-------------------+---------+------------------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|      Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|         Complemento|             Bairro|      Cep|           Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------------+--------------------+-------------------+--------------------+----------+--------------------+-------------------+---------+------------------+--------------+--------------+---------------+-----------------+--------+
|            SE|            SP|       SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|       306|                null|     JARDIM ZULMIRA|18061-000|          GASOLINA|    24/03/2025|   

In [189]:
df_fuels.schema

StructType([StructField('Regiao - Sigla', StringType(), True), StructField('Estado - Sigla', StringType(), True), StructField('Municipio', StringType(), True), StructField('Revenda', StringType(), True), StructField('CNPJ da Revenda', StringType(), True), StructField('Nome da Rua', StringType(), True), StructField('Numero Rua', StringType(), True), StructField('Complemento', StringType(), True), StructField('Bairro', StringType(), True), StructField('Cep', StringType(), True), StructField('Produto', StringType(), True), StructField('Data da Coleta', StringType(), True), StructField('Valor de Venda', StringType(), True), StructField('Valor de Compra', DoubleType(), True), StructField('Unidade de Medida', StringType(), True), StructField('Bandeira', StringType(), True)])

In [190]:
df_fuels = df_fuels.select("Regiao - Sigla", "Estado - Sigla", "Municipio", "Revenda", "Produto", "Valor de Venda", "Valor de Compra", "Bandeira")
df_fuels.show()

+--------------+--------------+---------------+--------------------+------------------+--------------+---------------+--------+
|Regiao - Sigla|Estado - Sigla|      Municipio|             Revenda|           Produto|Valor de Venda|Valor de Compra|Bandeira|
+--------------+--------------+---------------+--------------------+------------------+--------------+---------------+--------+
|            SE|            SP|       SOROCABA|COMPETRO COMERCIO...|          GASOLINA|          5,65|           null|  BRANCA|
|            SE|            SP|       SOROCABA|COMPETRO COMERCIO...|            ETANOL|          3,78|           null|  BRANCA|
|             N|            AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|          GASOLINA|          8,09|           null|IPIRANGA|
|             N|            AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|GASOLINA ADITIVADA|          8,19|           null|IPIRANGA|
|             N|            AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|            ETANOL|          5,95|  

In [191]:
df_fuels = df_fuels.withColumnRenamed("Regiao - Sigla", "Region") \
    .withColumnRenamed("Estado - Sigla", "State") \
    .withColumnRenamed("Municipio", "City") \
    .withColumnRenamed("Revenda", "Gas Station") \
    .withColumnRenamed("Produto", "Product") \
    .withColumnRenamed("Valor de Venda", "Sales Price") \
    .withColumnRenamed("Valor de Compra", "Purchase Price") \
    .withColumnRenamed("Bandeira", "Fuel Brand")
df_fuels.show()

+------+-----+---------------+--------------------+------------------+-----------+--------------+----------+
|Region|State|           City|         Gas Station|           Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+---------------+--------------------+------------------+-----------+--------------+----------+
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|          GASOLINA|       5,65|          null|    BRANCA|
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|            ETANOL|       3,78|          null|    BRANCA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|          GASOLINA|       8,09|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|GASOLINA ADITIVADA|       8,19|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|            ETANOL|       5,95|          null|  IPIRANGA|
|     N|   AC|     RIO BRANCO|CODEPE COMBUSTIVE...|          GASOLINA|       7,62|          null|     VIBRA|
|     N|   AC|     

In [192]:
df_fuels = df_fuels.withColumn(
    "Product", 
    when(df_fuels["Product"] == "ETANOL", "Ethanol")
    .when(df_fuels["Product"] == "GASOLINA", "Gasoline")
    .when(df_fuels["Product"] == "GASOLINA ADITIVADA", "Premium Gasoline")
    .when(df_fuels["Product"] == "DIESEL", "Diesel")
    .when(df_fuels["Product"] == "DIESEL S10", "Diesel S10")
    .when(df_fuels["Product"] == "GLP", "LPG")
    .when(df_fuels["Product"] == "GNV", "CNG")
    .otherwise(df_fuels["Product"])
)
df_fuels.show()

+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|Region|State|           City|         Gas Station|         Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|        Gasoline|       5,65|          null|    BRANCA|
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|         Ethanol|       3,78|          null|    BRANCA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|        Gasoline|       8,09|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Premium Gasoline|       8,19|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|         Ethanol|       5,95|          null|  IPIRANGA|
|     N|   AC|     RIO BRANCO|CODEPE COMBUSTIVE...|        Gasoline|       7,62|          null|     VIBRA|
|     N|   AC|     RIO BRANCO|CODEPE 

In [193]:
df_fuels = df_fuels.withColumn("Sales Price", regexp_replace("Sales Price", ",", "."))
df_fuels.show(10)

+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|Region|State|           City|         Gas Station|         Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|        Gasoline|       5.65|          null|    BRANCA|
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|         Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|        Gasoline|       8.09|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Premium Gasoline|       8.19|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|         Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|     RIO BRANCO|CODEPE COMBUSTIVE...|        Gasoline|       7.62|          null|     VIBRA|
|     N|   AC|     RIO BRANCO|CODEPE 

In [194]:
df_fuels = df_fuels.withColumn("Sales Price", col("Sales Price").cast(FloatType()))
df_fuels.show(10)

+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|Region|State|           City|         Gas Station|         Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|        Gasoline|       5.65|          null|    BRANCA|
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|         Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|        Gasoline|       8.09|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Premium Gasoline|       8.19|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|         Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|     RIO BRANCO|CODEPE COMBUSTIVE...|        Gasoline|       7.62|          null|     VIBRA|
|     N|   AC|     RIO BRANCO|CODEPE 

In [195]:
df_fuels.filter(col("Sales Price").isNull()).show()

+------+-----+----+-----------+-------+-----------+--------------+----------+
|Region|State|City|Gas Station|Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+----+-----------+-------+-----------+--------------+----------+
+------+-----+----+-----------+-------+-----------+--------------+----------+



# National Average Price

In [196]:
df_fuels.groupBy("Product") \
    .agg(round(avg("Sales Price"), 2).alias("Average Sales Price")) \
    .orderBy('Average Sales Price', ascending=False) \
    .show()

+----------------+-------------------+
|         Product|Average Sales Price|
+----------------+-------------------+
|             LPG|             108.58|
|Premium Gasoline|               6.52|
|      Diesel S10|                6.4|
|          Diesel|               6.35|
|        Gasoline|               6.33|
|             CNG|               4.82|
|         Ethanol|               4.52|
+----------------+-------------------+



# Ethanol Prices

In [197]:
df_ethanol = df_fuels.filter(df_fuels["Product"] == "Ethanol")
df_ethanol.show()

+------+-----+-------------------+--------------------+-------+-----------+--------------+----------+
|Region|State|               City|         Gas Station|Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+-------------------+--------------------+-------+-----------+--------------+----------+
|    SE|   SP|           SOROCABA|COMPETRO COMERCIO...|Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|    CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|         RIO BRANCO|CODEPE COMBUSTIVE...|Ethanol|       5.29|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO CASTAN...|Ethanol|       5.49|          null|   EQUADOR|
|     N|   AC|         RIO BRANCO|AUTO POSTO CIDADE...|Ethanol|       5.29|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO DISTRI...|Ethanol|       5.49|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO SANTO ...|Ethanol|       5.32|       

### Ethanol Prices by Region

In [198]:
df_ethanol.groupBy("Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price")) \
    .orderBy("Average Ethanol Sales Price", ascending=False) \
    .show()

+------+---------------------------+
|Region|Average Ethanol Sales Price|
+------+---------------------------+
|     N|                       5.11|
|    NE|                       4.91|
|     S|                        4.7|
|    SE|                        4.3|
|    CO|                       4.28|
+------+---------------------------+



### Ethanol Prices by State

In [199]:
ethanol_by_state = df_ethanol.groupBy("State") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [200]:
ethanol_by_state.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+-----+---------------------------+
|State|Average Ethanol Sales Price|
+-----+---------------------------+
|   AC|                       5.57|
|   AM|                       5.48|
|   AP|                       5.47|
|   RO|                       5.25|
|   CE|                       5.25|
|   RR|                       5.16|
|   MA|                       5.04|
|   AL|                       4.98|
|   RN|                       4.96|
|   RS|                       4.94|
+-----+---------------------------+
only showing top 10 rows



In [201]:
ethanol_by_state.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+-----+---------------------------+
|State|Average Ethanol Sales Price|
+-----+---------------------------+
|   MT|                       4.09|
|   SP|                       4.16|
|   MS|                        4.2|
|   GO|                       4.32|
|   MG|                       4.38|
|   PR|                       4.53|
|   ES|                       4.59|
|   PB|                       4.62|
|   RJ|                       4.69|
|   DF|                       4.71|
+-----+---------------------------+
only showing top 10 rows



### Ethanol Prices by City

In [202]:
ethanol_by_city = df_ethanol.groupBy("City", "State", "Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [203]:
ethanol_by_city.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+-----+------+---------------------------+
|                City|State|Region|Average Ethanol Sales Price|
+--------------------+-----+------+---------------------------+
|     CRUZEIRO DO SUL|   AC|     N|                       5.98|
|             VILHENA|   RO|     N|                       5.74|
|SANTANA DO LIVRAM...|   RS|     S|                       5.68|
|            ALTAMIRA|   PA|     N|                       5.61|
|           ITAPIPOCA|   CE|    NE|                        5.6|
|                CODO|   MA|    NE|                       5.57|
|              MANAUS|   AM|     N|                       5.49|
|              MACAPA|   AP|     N|                       5.47|
|             CRATEUS|   CE|    NE|                       5.47|
|         PORTO VELHO|   RO|     N|                       5.46|
+--------------------+-----+------+---------------------------+
only showing top 10 rows



In [204]:
ethanol_by_city.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+--------------------+-----+------+---------------------------+
|                City|State|Region|Average Ethanol Sales Price|
+--------------------+-----+------+---------------------------+
|                LINS|   SP|    SE|                       3.79|
|           ARACATUBA|   SP|    SE|                       3.79|
|SAO JOSE DO RIO P...|   SP|    SE|                       3.83|
|             BIRIGUI|   SP|    SE|                       3.85|
|            ANAPOLIS|   GO|    CO|                       3.87|
|            GOIATUBA|   GO|    CO|                       3.93|
|             MARILIA|   SP|    SE|                       3.94|
|          UBERLANDIA|   MG|    SE|                       3.94|
|              FRANCA|   SP|    SE|                       3.94|
|          ADAMANTINA|   SP|    SE|                       3.96|
+--------------------+-----+------+---------------------------+
only showing top 10 rows



### Ethanol Prices by Brand

In [205]:
ethanol_by_brand = df_fuels.filter(col("Product") == "Ethanol") \
    .groupBy("Fuel Brand") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [206]:
ethanol_by_brand.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+-----------------+---------------------------+
|       Fuel Brand|Average Ethanol Sales Price|
+-----------------+---------------------------+
|RZD DISTRIBUIDORA|                       5.49|
|          ATEM' S|                       5.47|
|          EQUADOR|                       5.35|
|              FAN|                       5.22|
|               SP|                       5.21|
|            SABBÁ|                       5.02|
|      RAIZEN MIME|                       4.99|
|      SANTA LUCIA|                       4.99|
|           DISLUB|                       4.96|
|         ON PETRO|                       4.93|
+-----------------+---------------------------+
only showing top 10 rows



In [207]:
ethanol_by_brand.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+----------------+---------------------------+
|      Fuel Brand|Average Ethanol Sales Price|
+----------------+---------------------------+
|       ROYAL FIC|                       3.79|
|          D`MAIS|                       4.03|
|SIM DISTRIBUIDOR|                       4.04|
|            WATT|                       4.16|
|           IDAZA|                       4.25|
|        CIAPETRO|                       4.27|
|          TAURUS|                       4.31|
|   TOTALENERGIES|                       4.37|
|      PETROSERRA|                        4.4|
|          BRANCA|                       4.41|
+----------------+---------------------------+
only showing top 10 rows



### Ethanol Prices by Gas Station

In [208]:
df_fuels.show()

+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|Region|State|           City|         Gas Station|         Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|        Gasoline|       5.65|          null|    BRANCA|
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|         Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|        Gasoline|       8.09|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Premium Gasoline|       8.19|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|         Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|     RIO BRANCO|CODEPE COMBUSTIVE...|        Gasoline|       7.62|          null|     VIBRA|
|     N|   AC|     RIO BRANCO|CODEPE 

In [211]:
ethanol_by_gas_station = df_fuels.filter(col("Product") == "Ethanol") \
    .groupBy("Gas Station", "City", "State", "Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [212]:
ethanol_by_gas_station.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+---------------+-----+------+---------------------------+
|         Gas Station|           City|State|Region|Average Ethanol Sales Price|
+--------------------+---------------+-----+------+---------------------------+
|COMERCIAL ALTO SA...|  SERRA TALHADA|   PE|    NE|                       6.49|
|SADI DALLACORTE E...|        CHAPECO|   SC|     S|                       6.36|
|C M DERIVADOS DE ...|      FORTALEZA|   CE|    NE|                       6.09|
| S & G PETROLEO LTDA|CRUZEIRO DO SUL|   AC|     N|                       6.08|
|PG COMERCIO E TRA...|        VACARIA|   RS|     S|                       6.05|
| SKY AUTO POSTO LTDA|      SAO PAULO|   SP|    SE|                       5.99|
|AUTO POSTO DUQUE ...|      SAO PAULO|   SP|    SE|                       5.99|
|MASTER POSTO COME...|         SOBRAL|   CE|    NE|                       5.99|
|AUTO POSTO R&R AL...|        BARUERI|   SP|    SE|                       5.99|
|AUTO POSTO CATARI...|        VILHENA|  

In [213]:
ethanol_by_gas_station.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+---------------+-----+------+---------------------------+
|         Gas Station|           City|State|Region|Average Ethanol Sales Price|
+--------------------+---------------+-----+------+---------------------------+
|COMERCIAL ALTO SA...|  SERRA TALHADA|   PE|    NE|                       6.49|
|SADI DALLACORTE E...|        CHAPECO|   SC|     S|                       6.36|
|C M DERIVADOS DE ...|      FORTALEZA|   CE|    NE|                       6.09|
| S & G PETROLEO LTDA|CRUZEIRO DO SUL|   AC|     N|                       6.08|
|PG COMERCIO E TRA...|        VACARIA|   RS|     S|                       6.05|
| SKY AUTO POSTO LTDA|      SAO PAULO|   SP|    SE|                       5.99|
|AUTO POSTO DUQUE ...|      SAO PAULO|   SP|    SE|                       5.99|
|MASTER POSTO COME...|         SOBRAL|   CE|    NE|                       5.99|
|AUTO POSTO R&R AL...|        BARUERI|   SP|    SE|                       5.99|
|AUTO POSTO CATARI...|        VILHENA|  

# Gasoline Prices

In [ ]:
df_gasoline = df_fuels.filter(df_fuels["Product"] == "Gasoline")
df_gasoline.show()

+------+-----+-------------------+--------------------+-------+-----------+--------------+----------+
|Region|State|               City|         Gas Station|Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+-------------------+--------------------+-------+-----------+--------------+----------+
|    SE|   SP|           SOROCABA|COMPETRO COMERCIO...|Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|    CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|         RIO BRANCO|CODEPE COMBUSTIVE...|Ethanol|       5.29|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO CASTAN...|Ethanol|       5.49|          null|   EQUADOR|
|     N|   AC|         RIO BRANCO|AUTO POSTO CIDADE...|Ethanol|       5.29|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO DISTRI...|Ethanol|       5.49|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO SANTO ...|Ethanol|       5.32|       

### Ethanol Prices by Region

In [ ]:
df_gasoline.groupBy("Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Gasoline Sales Price")) \
    .orderBy("Average Gasoline Sales Price", ascending=False) \
    .show()

+------+---------------------------+
|Region|Average Ethanol Sales Price|
+------+---------------------------+
|     N|                       5.11|
|    NE|                       4.91|
|     S|                        4.7|
|    SE|                        4.3|
|    CO|                       4.28|
+------+---------------------------+



### Gasoline Prices by State

In [ ]:
gasoline_by_state = df_gasoline.groupBy("State") \
    .agg(round(avg("Sales Price"), 2).alias("Average Gasoline Sales Price"))

In [ ]:
gasoline_by_state.orderBy("Average Gasoline Sales Price", ascending=False).show(10)

+-----+---------------------------+
|State|Average Ethanol Sales Price|
+-----+---------------------------+
|   AC|                       5.57|
|   AM|                       5.48|
|   AP|                       5.47|
|   RO|                       5.25|
|   CE|                       5.25|
|   RR|                       5.16|
|   MA|                       5.04|
|   AL|                       4.98|
|   RN|                       4.96|
|   RS|                       4.94|
+-----+---------------------------+
only showing top 10 rows



In [ ]:
gasoline_by_state.orderBy("Average Gasoline Sales Price", ascending=True).show(10)

+-----+---------------------------+
|State|Average Ethanol Sales Price|
+-----+---------------------------+
|   MT|                       4.09|
|   SP|                       4.16|
|   MS|                        4.2|
|   GO|                       4.32|
|   MG|                       4.38|
|   PR|                       4.53|
|   ES|                       4.59|
|   PB|                       4.62|
|   RJ|                       4.69|
|   DF|                       4.71|
+-----+---------------------------+
only showing top 10 rows



### Gasoline Prices by City

In [ ]:
gasoline_by_city = df_gasoline.groupBy("City", "State", "Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Gasoline Sales Price"))

In [ ]:
gasoline_by_city.orderBy("Average Gasoline Sales Price", ascending=False).show(10)

+--------------------+-----+------+---------------------------+
|                City|State|Region|Average Ethanol Sales Price|
+--------------------+-----+------+---------------------------+
|     CRUZEIRO DO SUL|   AC|     N|                       5.98|
|             VILHENA|   RO|     N|                       5.74|
|SANTANA DO LIVRAM...|   RS|     S|                       5.68|
|            ALTAMIRA|   PA|     N|                       5.61|
|           ITAPIPOCA|   CE|    NE|                        5.6|
|                CODO|   MA|    NE|                       5.57|
|              MANAUS|   AM|     N|                       5.49|
|              MACAPA|   AP|     N|                       5.47|
|             CRATEUS|   CE|    NE|                       5.47|
|         PORTO VELHO|   RO|     N|                       5.46|
+--------------------+-----+------+---------------------------+
only showing top 10 rows



In [ ]:
gasoline_by_city.orderBy("Average Gasoline Sales Price", ascending=True).show(10)

+--------------------+-----+------+---------------------------+
|                City|State|Region|Average Ethanol Sales Price|
+--------------------+-----+------+---------------------------+
|                LINS|   SP|    SE|                       3.79|
|           ARACATUBA|   SP|    SE|                       3.79|
|SAO JOSE DO RIO P...|   SP|    SE|                       3.83|
|             BIRIGUI|   SP|    SE|                       3.85|
|            ANAPOLIS|   GO|    CO|                       3.87|
|            GOIATUBA|   GO|    CO|                       3.93|
|             MARILIA|   SP|    SE|                       3.94|
|          UBERLANDIA|   MG|    SE|                       3.94|
|              FRANCA|   SP|    SE|                       3.94|
|          ADAMANTINA|   SP|    SE|                       3.96|
+--------------------+-----+------+---------------------------+
only showing top 10 rows



### Gasoline Prices by Brand

In [ ]:
gasoline_by_brand = df_fuels.filter(col("Product") == "Gasoline") \
    .groupBy("Fuel Brand") \
    .agg(round(avg("Sales Price"), 2).alias("Average Gasoline Sales Price"))

In [ ]:
ethanol_by_brand.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+-----------------+---------------------------+
|       Fuel Brand|Average Ethanol Sales Price|
+-----------------+---------------------------+
|RZD DISTRIBUIDORA|                       5.49|
|          ATEM' S|                       5.47|
|          EQUADOR|                       5.35|
|              FAN|                       5.22|
|               SP|                       5.21|
|            SABBÁ|                       5.02|
|      RAIZEN MIME|                       4.99|
|      SANTA LUCIA|                       4.99|
|           DISLUB|                       4.96|
|         ON PETRO|                       4.93|
+-----------------+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_brand.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+----------------+---------------------------+
|      Fuel Brand|Average Ethanol Sales Price|
+----------------+---------------------------+
|       ROYAL FIC|                       3.79|
|          D`MAIS|                       4.03|
|SIM DISTRIBUIDOR|                       4.04|
|            WATT|                       4.16|
|           IDAZA|                       4.25|
|        CIAPETRO|                       4.27|
|          TAURUS|                       4.31|
|   TOTALENERGIES|                       4.37|
|      PETROSERRA|                        4.4|
|          BRANCA|                       4.41|
+----------------+---------------------------+
only showing top 10 rows



### Ethanol Prices by Gas Station

In [ ]:
df_fuels.show()

+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|Region|State|           City|         Gas Station|         Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|        Gasoline|       5.65|          null|    BRANCA|
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|         Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|        Gasoline|       8.09|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Premium Gasoline|       8.19|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|         Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|     RIO BRANCO|CODEPE COMBUSTIVE...|        Gasoline|       7.62|          null|     VIBRA|
|     N|   AC|     RIO BRANCO|CODEPE 

In [ ]:
ethanol_by_gas_station = df_fuels.filter(col("Product") == "Ethanol") \
    .groupBy("Gas Station", "City", "State", "Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_gas_station.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+---------------+-----+------+---------------------------+
|         Gas Station|           City|State|Region|Average Ethanol Sales Price|
+--------------------+---------------+-----+------+---------------------------+
|COMERCIAL ALTO SA...|  SERRA TALHADA|   PE|    NE|                       6.49|
|SADI DALLACORTE E...|        CHAPECO|   SC|     S|                       6.36|
|C M DERIVADOS DE ...|      FORTALEZA|   CE|    NE|                       6.09|
| S & G PETROLEO LTDA|CRUZEIRO DO SUL|   AC|     N|                       6.08|
|PG COMERCIO E TRA...|        VACARIA|   RS|     S|                       6.05|
| SKY AUTO POSTO LTDA|      SAO PAULO|   SP|    SE|                       5.99|
|AUTO POSTO DUQUE ...|      SAO PAULO|   SP|    SE|                       5.99|
|MASTER POSTO COME...|         SOBRAL|   CE|    NE|                       5.99|
|AUTO POSTO R&R AL...|        BARUERI|   SP|    SE|                       5.99|
|AUTO POSTO CATARI...|        VILHENA|  

In [ ]:
ethanol_by_gas_station.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+---------------+-----+------+---------------------------+
|         Gas Station|           City|State|Region|Average Ethanol Sales Price|
+--------------------+---------------+-----+------+---------------------------+
|COMERCIAL ALTO SA...|  SERRA TALHADA|   PE|    NE|                       6.49|
|SADI DALLACORTE E...|        CHAPECO|   SC|     S|                       6.36|
|C M DERIVADOS DE ...|      FORTALEZA|   CE|    NE|                       6.09|
| S & G PETROLEO LTDA|CRUZEIRO DO SUL|   AC|     N|                       6.08|
|PG COMERCIO E TRA...|        VACARIA|   RS|     S|                       6.05|
| SKY AUTO POSTO LTDA|      SAO PAULO|   SP|    SE|                       5.99|
|AUTO POSTO DUQUE ...|      SAO PAULO|   SP|    SE|                       5.99|
|MASTER POSTO COME...|         SOBRAL|   CE|    NE|                       5.99|
|AUTO POSTO R&R AL...|        BARUERI|   SP|    SE|                       5.99|
|AUTO POSTO CATARI...|        VILHENA|  

# Premium Gasoline Prices

In [ ]:
df_ethanol = df_fuels.filter(df_fuels["Product"] == "Ethanol")
df_ethanol.show()

+------+-----+-------------------+--------------------+-------+-----------+--------------+----------+
|Region|State|               City|         Gas Station|Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+-------------------+--------------------+-------+-----------+--------------+----------+
|    SE|   SP|           SOROCABA|COMPETRO COMERCIO...|Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|    CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|         RIO BRANCO|CODEPE COMBUSTIVE...|Ethanol|       5.29|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO CASTAN...|Ethanol|       5.49|          null|   EQUADOR|
|     N|   AC|         RIO BRANCO|AUTO POSTO CIDADE...|Ethanol|       5.29|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO DISTRI...|Ethanol|       5.49|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO SANTO ...|Ethanol|       5.32|       

### Ethanol Prices by Region

In [ ]:
df_ethanol.groupBy("Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price")) \
    .orderBy("Average Ethanol Sales Price", ascending=False) \
    .show()

+------+---------------------------+
|Region|Average Ethanol Sales Price|
+------+---------------------------+
|     N|                       5.11|
|    NE|                       4.91|
|     S|                        4.7|
|    SE|                        4.3|
|    CO|                       4.28|
+------+---------------------------+



### Ethanol Prices by State

In [ ]:
ethanol_by_state = df_ethanol.groupBy("State") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_state.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+-----+---------------------------+
|State|Average Ethanol Sales Price|
+-----+---------------------------+
|   AC|                       5.57|
|   AM|                       5.48|
|   AP|                       5.47|
|   RO|                       5.25|
|   CE|                       5.25|
|   RR|                       5.16|
|   MA|                       5.04|
|   AL|                       4.98|
|   RN|                       4.96|
|   RS|                       4.94|
+-----+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_state.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+-----+---------------------------+
|State|Average Ethanol Sales Price|
+-----+---------------------------+
|   MT|                       4.09|
|   SP|                       4.16|
|   MS|                        4.2|
|   GO|                       4.32|
|   MG|                       4.38|
|   PR|                       4.53|
|   ES|                       4.59|
|   PB|                       4.62|
|   RJ|                       4.69|
|   DF|                       4.71|
+-----+---------------------------+
only showing top 10 rows



### Ethanol Prices by City

In [ ]:
ethanol_by_city = df_ethanol.groupBy("City", "State", "Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_city.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+-----+------+---------------------------+
|                City|State|Region|Average Ethanol Sales Price|
+--------------------+-----+------+---------------------------+
|     CRUZEIRO DO SUL|   AC|     N|                       5.98|
|             VILHENA|   RO|     N|                       5.74|
|SANTANA DO LIVRAM...|   RS|     S|                       5.68|
|            ALTAMIRA|   PA|     N|                       5.61|
|           ITAPIPOCA|   CE|    NE|                        5.6|
|                CODO|   MA|    NE|                       5.57|
|              MANAUS|   AM|     N|                       5.49|
|              MACAPA|   AP|     N|                       5.47|
|             CRATEUS|   CE|    NE|                       5.47|
|         PORTO VELHO|   RO|     N|                       5.46|
+--------------------+-----+------+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_city.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+--------------------+-----+------+---------------------------+
|                City|State|Region|Average Ethanol Sales Price|
+--------------------+-----+------+---------------------------+
|                LINS|   SP|    SE|                       3.79|
|           ARACATUBA|   SP|    SE|                       3.79|
|SAO JOSE DO RIO P...|   SP|    SE|                       3.83|
|             BIRIGUI|   SP|    SE|                       3.85|
|            ANAPOLIS|   GO|    CO|                       3.87|
|            GOIATUBA|   GO|    CO|                       3.93|
|             MARILIA|   SP|    SE|                       3.94|
|          UBERLANDIA|   MG|    SE|                       3.94|
|              FRANCA|   SP|    SE|                       3.94|
|          ADAMANTINA|   SP|    SE|                       3.96|
+--------------------+-----+------+---------------------------+
only showing top 10 rows



### Ethanol Prices by Brand

In [ ]:
ethanol_by_brand = df_fuels.filter(col("Product") == "Ethanol") \
    .groupBy("Fuel Brand") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_brand.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+-----------------+---------------------------+
|       Fuel Brand|Average Ethanol Sales Price|
+-----------------+---------------------------+
|RZD DISTRIBUIDORA|                       5.49|
|          ATEM' S|                       5.47|
|          EQUADOR|                       5.35|
|              FAN|                       5.22|
|               SP|                       5.21|
|            SABBÁ|                       5.02|
|      RAIZEN MIME|                       4.99|
|      SANTA LUCIA|                       4.99|
|           DISLUB|                       4.96|
|         ON PETRO|                       4.93|
+-----------------+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_brand.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+----------------+---------------------------+
|      Fuel Brand|Average Ethanol Sales Price|
+----------------+---------------------------+
|       ROYAL FIC|                       3.79|
|          D`MAIS|                       4.03|
|SIM DISTRIBUIDOR|                       4.04|
|            WATT|                       4.16|
|           IDAZA|                       4.25|
|        CIAPETRO|                       4.27|
|          TAURUS|                       4.31|
|   TOTALENERGIES|                       4.37|
|      PETROSERRA|                        4.4|
|          BRANCA|                       4.41|
+----------------+---------------------------+
only showing top 10 rows



### Ethanol Prices by Gas Station

In [ ]:
df_fuels.show()

+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|Region|State|           City|         Gas Station|         Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|        Gasoline|       5.65|          null|    BRANCA|
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|         Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|        Gasoline|       8.09|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Premium Gasoline|       8.19|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|         Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|     RIO BRANCO|CODEPE COMBUSTIVE...|        Gasoline|       7.62|          null|     VIBRA|
|     N|   AC|     RIO BRANCO|CODEPE 

In [ ]:
ethanol_by_gas_station = df_fuels.filter(col("Product") == "Ethanol") \
    .groupBy("Gas Station", "City", "State", "Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_gas_station.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+---------------+-----+------+---------------------------+
|         Gas Station|           City|State|Region|Average Ethanol Sales Price|
+--------------------+---------------+-----+------+---------------------------+
|COMERCIAL ALTO SA...|  SERRA TALHADA|   PE|    NE|                       6.49|
|SADI DALLACORTE E...|        CHAPECO|   SC|     S|                       6.36|
|C M DERIVADOS DE ...|      FORTALEZA|   CE|    NE|                       6.09|
| S & G PETROLEO LTDA|CRUZEIRO DO SUL|   AC|     N|                       6.08|
|PG COMERCIO E TRA...|        VACARIA|   RS|     S|                       6.05|
| SKY AUTO POSTO LTDA|      SAO PAULO|   SP|    SE|                       5.99|
|AUTO POSTO DUQUE ...|      SAO PAULO|   SP|    SE|                       5.99|
|MASTER POSTO COME...|         SOBRAL|   CE|    NE|                       5.99|
|AUTO POSTO R&R AL...|        BARUERI|   SP|    SE|                       5.99|
|AUTO POSTO CATARI...|        VILHENA|  

In [ ]:
ethanol_by_gas_station.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+---------------+-----+------+---------------------------+
|         Gas Station|           City|State|Region|Average Ethanol Sales Price|
+--------------------+---------------+-----+------+---------------------------+
|COMERCIAL ALTO SA...|  SERRA TALHADA|   PE|    NE|                       6.49|
|SADI DALLACORTE E...|        CHAPECO|   SC|     S|                       6.36|
|C M DERIVADOS DE ...|      FORTALEZA|   CE|    NE|                       6.09|
| S & G PETROLEO LTDA|CRUZEIRO DO SUL|   AC|     N|                       6.08|
|PG COMERCIO E TRA...|        VACARIA|   RS|     S|                       6.05|
| SKY AUTO POSTO LTDA|      SAO PAULO|   SP|    SE|                       5.99|
|AUTO POSTO DUQUE ...|      SAO PAULO|   SP|    SE|                       5.99|
|MASTER POSTO COME...|         SOBRAL|   CE|    NE|                       5.99|
|AUTO POSTO R&R AL...|        BARUERI|   SP|    SE|                       5.99|
|AUTO POSTO CATARI...|        VILHENA|  

# Diesel Prices

In [ ]:
df_ethanol = df_fuels.filter(df_fuels["Product"] == "Ethanol")
df_ethanol.show()

+------+-----+-------------------+--------------------+-------+-----------+--------------+----------+
|Region|State|               City|         Gas Station|Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+-------------------+--------------------+-------+-----------+--------------+----------+
|    SE|   SP|           SOROCABA|COMPETRO COMERCIO...|Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|    CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|         RIO BRANCO|CODEPE COMBUSTIVE...|Ethanol|       5.29|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO CASTAN...|Ethanol|       5.49|          null|   EQUADOR|
|     N|   AC|         RIO BRANCO|AUTO POSTO CIDADE...|Ethanol|       5.29|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO DISTRI...|Ethanol|       5.49|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO SANTO ...|Ethanol|       5.32|       

### Ethanol Prices by Region

In [ ]:
df_ethanol.groupBy("Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price")) \
    .orderBy("Average Ethanol Sales Price", ascending=False) \
    .show()

+------+---------------------------+
|Region|Average Ethanol Sales Price|
+------+---------------------------+
|     N|                       5.11|
|    NE|                       4.91|
|     S|                        4.7|
|    SE|                        4.3|
|    CO|                       4.28|
+------+---------------------------+



### Ethanol Prices by State

In [ ]:
ethanol_by_state = df_ethanol.groupBy("State") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_state.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+-----+---------------------------+
|State|Average Ethanol Sales Price|
+-----+---------------------------+
|   AC|                       5.57|
|   AM|                       5.48|
|   AP|                       5.47|
|   RO|                       5.25|
|   CE|                       5.25|
|   RR|                       5.16|
|   MA|                       5.04|
|   AL|                       4.98|
|   RN|                       4.96|
|   RS|                       4.94|
+-----+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_state.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+-----+---------------------------+
|State|Average Ethanol Sales Price|
+-----+---------------------------+
|   MT|                       4.09|
|   SP|                       4.16|
|   MS|                        4.2|
|   GO|                       4.32|
|   MG|                       4.38|
|   PR|                       4.53|
|   ES|                       4.59|
|   PB|                       4.62|
|   RJ|                       4.69|
|   DF|                       4.71|
+-----+---------------------------+
only showing top 10 rows



### Ethanol Prices by City

In [ ]:
ethanol_by_city = df_ethanol.groupBy("City", "State", "Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_city.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+-----+------+---------------------------+
|                City|State|Region|Average Ethanol Sales Price|
+--------------------+-----+------+---------------------------+
|     CRUZEIRO DO SUL|   AC|     N|                       5.98|
|             VILHENA|   RO|     N|                       5.74|
|SANTANA DO LIVRAM...|   RS|     S|                       5.68|
|            ALTAMIRA|   PA|     N|                       5.61|
|           ITAPIPOCA|   CE|    NE|                        5.6|
|                CODO|   MA|    NE|                       5.57|
|              MANAUS|   AM|     N|                       5.49|
|              MACAPA|   AP|     N|                       5.47|
|             CRATEUS|   CE|    NE|                       5.47|
|         PORTO VELHO|   RO|     N|                       5.46|
+--------------------+-----+------+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_city.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+--------------------+-----+------+---------------------------+
|                City|State|Region|Average Ethanol Sales Price|
+--------------------+-----+------+---------------------------+
|                LINS|   SP|    SE|                       3.79|
|           ARACATUBA|   SP|    SE|                       3.79|
|SAO JOSE DO RIO P...|   SP|    SE|                       3.83|
|             BIRIGUI|   SP|    SE|                       3.85|
|            ANAPOLIS|   GO|    CO|                       3.87|
|            GOIATUBA|   GO|    CO|                       3.93|
|             MARILIA|   SP|    SE|                       3.94|
|          UBERLANDIA|   MG|    SE|                       3.94|
|              FRANCA|   SP|    SE|                       3.94|
|          ADAMANTINA|   SP|    SE|                       3.96|
+--------------------+-----+------+---------------------------+
only showing top 10 rows



### Ethanol Prices by Brand

In [ ]:
ethanol_by_brand = df_fuels.filter(col("Product") == "Ethanol") \
    .groupBy("Fuel Brand") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_brand.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+-----------------+---------------------------+
|       Fuel Brand|Average Ethanol Sales Price|
+-----------------+---------------------------+
|RZD DISTRIBUIDORA|                       5.49|
|          ATEM' S|                       5.47|
|          EQUADOR|                       5.35|
|              FAN|                       5.22|
|               SP|                       5.21|
|            SABBÁ|                       5.02|
|      RAIZEN MIME|                       4.99|
|      SANTA LUCIA|                       4.99|
|           DISLUB|                       4.96|
|         ON PETRO|                       4.93|
+-----------------+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_brand.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+----------------+---------------------------+
|      Fuel Brand|Average Ethanol Sales Price|
+----------------+---------------------------+
|       ROYAL FIC|                       3.79|
|          D`MAIS|                       4.03|
|SIM DISTRIBUIDOR|                       4.04|
|            WATT|                       4.16|
|           IDAZA|                       4.25|
|        CIAPETRO|                       4.27|
|          TAURUS|                       4.31|
|   TOTALENERGIES|                       4.37|
|      PETROSERRA|                        4.4|
|          BRANCA|                       4.41|
+----------------+---------------------------+
only showing top 10 rows



### Ethanol Prices by Gas Station

In [ ]:
df_fuels.show()

+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|Region|State|           City|         Gas Station|         Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|        Gasoline|       5.65|          null|    BRANCA|
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|         Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|        Gasoline|       8.09|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Premium Gasoline|       8.19|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|         Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|     RIO BRANCO|CODEPE COMBUSTIVE...|        Gasoline|       7.62|          null|     VIBRA|
|     N|   AC|     RIO BRANCO|CODEPE 

In [ ]:
ethanol_by_gas_station = df_fuels.filter(col("Product") == "Ethanol") \
    .groupBy("Gas Station", "City", "State", "Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_gas_station.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+---------------+-----+------+---------------------------+
|         Gas Station|           City|State|Region|Average Ethanol Sales Price|
+--------------------+---------------+-----+------+---------------------------+
|COMERCIAL ALTO SA...|  SERRA TALHADA|   PE|    NE|                       6.49|
|SADI DALLACORTE E...|        CHAPECO|   SC|     S|                       6.36|
|C M DERIVADOS DE ...|      FORTALEZA|   CE|    NE|                       6.09|
| S & G PETROLEO LTDA|CRUZEIRO DO SUL|   AC|     N|                       6.08|
|PG COMERCIO E TRA...|        VACARIA|   RS|     S|                       6.05|
| SKY AUTO POSTO LTDA|      SAO PAULO|   SP|    SE|                       5.99|
|AUTO POSTO DUQUE ...|      SAO PAULO|   SP|    SE|                       5.99|
|MASTER POSTO COME...|         SOBRAL|   CE|    NE|                       5.99|
|AUTO POSTO R&R AL...|        BARUERI|   SP|    SE|                       5.99|
|AUTO POSTO CATARI...|        VILHENA|  

In [ ]:
ethanol_by_gas_station.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+---------------+-----+------+---------------------------+
|         Gas Station|           City|State|Region|Average Ethanol Sales Price|
+--------------------+---------------+-----+------+---------------------------+
|COMERCIAL ALTO SA...|  SERRA TALHADA|   PE|    NE|                       6.49|
|SADI DALLACORTE E...|        CHAPECO|   SC|     S|                       6.36|
|C M DERIVADOS DE ...|      FORTALEZA|   CE|    NE|                       6.09|
| S & G PETROLEO LTDA|CRUZEIRO DO SUL|   AC|     N|                       6.08|
|PG COMERCIO E TRA...|        VACARIA|   RS|     S|                       6.05|
| SKY AUTO POSTO LTDA|      SAO PAULO|   SP|    SE|                       5.99|
|AUTO POSTO DUQUE ...|      SAO PAULO|   SP|    SE|                       5.99|
|MASTER POSTO COME...|         SOBRAL|   CE|    NE|                       5.99|
|AUTO POSTO R&R AL...|        BARUERI|   SP|    SE|                       5.99|
|AUTO POSTO CATARI...|        VILHENA|  

# Diesel S10 Prices

In [ ]:
df_ethanol = df_fuels.filter(df_fuels["Product"] == "Ethanol")
df_ethanol.show()

+------+-----+-------------------+--------------------+-------+-----------+--------------+----------+
|Region|State|               City|         Gas Station|Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+-------------------+--------------------+-------+-----------+--------------+----------+
|    SE|   SP|           SOROCABA|COMPETRO COMERCIO...|Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|    CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|         RIO BRANCO|CODEPE COMBUSTIVE...|Ethanol|       5.29|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO CASTAN...|Ethanol|       5.49|          null|   EQUADOR|
|     N|   AC|         RIO BRANCO|AUTO POSTO CIDADE...|Ethanol|       5.29|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO DISTRI...|Ethanol|       5.49|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO SANTO ...|Ethanol|       5.32|       

### Ethanol Prices by Region

In [ ]:
df_ethanol.groupBy("Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price")) \
    .orderBy("Average Ethanol Sales Price", ascending=False) \
    .show()

+------+---------------------------+
|Region|Average Ethanol Sales Price|
+------+---------------------------+
|     N|                       5.11|
|    NE|                       4.91|
|     S|                        4.7|
|    SE|                        4.3|
|    CO|                       4.28|
+------+---------------------------+



### Ethanol Prices by State

In [ ]:
ethanol_by_state = df_ethanol.groupBy("State") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_state.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+-----+---------------------------+
|State|Average Ethanol Sales Price|
+-----+---------------------------+
|   AC|                       5.57|
|   AM|                       5.48|
|   AP|                       5.47|
|   RO|                       5.25|
|   CE|                       5.25|
|   RR|                       5.16|
|   MA|                       5.04|
|   AL|                       4.98|
|   RN|                       4.96|
|   RS|                       4.94|
+-----+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_state.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+-----+---------------------------+
|State|Average Ethanol Sales Price|
+-----+---------------------------+
|   MT|                       4.09|
|   SP|                       4.16|
|   MS|                        4.2|
|   GO|                       4.32|
|   MG|                       4.38|
|   PR|                       4.53|
|   ES|                       4.59|
|   PB|                       4.62|
|   RJ|                       4.69|
|   DF|                       4.71|
+-----+---------------------------+
only showing top 10 rows



### Ethanol Prices by City

In [ ]:
ethanol_by_city = df_ethanol.groupBy("City", "State", "Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_city.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+-----+------+---------------------------+
|                City|State|Region|Average Ethanol Sales Price|
+--------------------+-----+------+---------------------------+
|     CRUZEIRO DO SUL|   AC|     N|                       5.98|
|             VILHENA|   RO|     N|                       5.74|
|SANTANA DO LIVRAM...|   RS|     S|                       5.68|
|            ALTAMIRA|   PA|     N|                       5.61|
|           ITAPIPOCA|   CE|    NE|                        5.6|
|                CODO|   MA|    NE|                       5.57|
|              MANAUS|   AM|     N|                       5.49|
|              MACAPA|   AP|     N|                       5.47|
|             CRATEUS|   CE|    NE|                       5.47|
|         PORTO VELHO|   RO|     N|                       5.46|
+--------------------+-----+------+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_city.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+--------------------+-----+------+---------------------------+
|                City|State|Region|Average Ethanol Sales Price|
+--------------------+-----+------+---------------------------+
|                LINS|   SP|    SE|                       3.79|
|           ARACATUBA|   SP|    SE|                       3.79|
|SAO JOSE DO RIO P...|   SP|    SE|                       3.83|
|             BIRIGUI|   SP|    SE|                       3.85|
|            ANAPOLIS|   GO|    CO|                       3.87|
|            GOIATUBA|   GO|    CO|                       3.93|
|             MARILIA|   SP|    SE|                       3.94|
|          UBERLANDIA|   MG|    SE|                       3.94|
|              FRANCA|   SP|    SE|                       3.94|
|          ADAMANTINA|   SP|    SE|                       3.96|
+--------------------+-----+------+---------------------------+
only showing top 10 rows



### Ethanol Prices by Brand

In [ ]:
ethanol_by_brand = df_fuels.filter(col("Product") == "Ethanol") \
    .groupBy("Fuel Brand") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_brand.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+-----------------+---------------------------+
|       Fuel Brand|Average Ethanol Sales Price|
+-----------------+---------------------------+
|RZD DISTRIBUIDORA|                       5.49|
|          ATEM' S|                       5.47|
|          EQUADOR|                       5.35|
|              FAN|                       5.22|
|               SP|                       5.21|
|            SABBÁ|                       5.02|
|      RAIZEN MIME|                       4.99|
|      SANTA LUCIA|                       4.99|
|           DISLUB|                       4.96|
|         ON PETRO|                       4.93|
+-----------------+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_brand.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+----------------+---------------------------+
|      Fuel Brand|Average Ethanol Sales Price|
+----------------+---------------------------+
|       ROYAL FIC|                       3.79|
|          D`MAIS|                       4.03|
|SIM DISTRIBUIDOR|                       4.04|
|            WATT|                       4.16|
|           IDAZA|                       4.25|
|        CIAPETRO|                       4.27|
|          TAURUS|                       4.31|
|   TOTALENERGIES|                       4.37|
|      PETROSERRA|                        4.4|
|          BRANCA|                       4.41|
+----------------+---------------------------+
only showing top 10 rows



### Ethanol Prices by Gas Station

In [ ]:
df_fuels.show()

+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|Region|State|           City|         Gas Station|         Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|        Gasoline|       5.65|          null|    BRANCA|
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|         Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|        Gasoline|       8.09|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Premium Gasoline|       8.19|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|         Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|     RIO BRANCO|CODEPE COMBUSTIVE...|        Gasoline|       7.62|          null|     VIBRA|
|     N|   AC|     RIO BRANCO|CODEPE 

In [ ]:
ethanol_by_gas_station = df_fuels.filter(col("Product") == "Ethanol") \
    .groupBy("Gas Station", "City", "State", "Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_gas_station.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+---------------+-----+------+---------------------------+
|         Gas Station|           City|State|Region|Average Ethanol Sales Price|
+--------------------+---------------+-----+------+---------------------------+
|COMERCIAL ALTO SA...|  SERRA TALHADA|   PE|    NE|                       6.49|
|SADI DALLACORTE E...|        CHAPECO|   SC|     S|                       6.36|
|C M DERIVADOS DE ...|      FORTALEZA|   CE|    NE|                       6.09|
| S & G PETROLEO LTDA|CRUZEIRO DO SUL|   AC|     N|                       6.08|
|PG COMERCIO E TRA...|        VACARIA|   RS|     S|                       6.05|
| SKY AUTO POSTO LTDA|      SAO PAULO|   SP|    SE|                       5.99|
|AUTO POSTO DUQUE ...|      SAO PAULO|   SP|    SE|                       5.99|
|MASTER POSTO COME...|         SOBRAL|   CE|    NE|                       5.99|
|AUTO POSTO R&R AL...|        BARUERI|   SP|    SE|                       5.99|
|AUTO POSTO CATARI...|        VILHENA|  

In [ ]:
ethanol_by_gas_station.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+---------------+-----+------+---------------------------+
|         Gas Station|           City|State|Region|Average Ethanol Sales Price|
+--------------------+---------------+-----+------+---------------------------+
|COMERCIAL ALTO SA...|  SERRA TALHADA|   PE|    NE|                       6.49|
|SADI DALLACORTE E...|        CHAPECO|   SC|     S|                       6.36|
|C M DERIVADOS DE ...|      FORTALEZA|   CE|    NE|                       6.09|
| S & G PETROLEO LTDA|CRUZEIRO DO SUL|   AC|     N|                       6.08|
|PG COMERCIO E TRA...|        VACARIA|   RS|     S|                       6.05|
| SKY AUTO POSTO LTDA|      SAO PAULO|   SP|    SE|                       5.99|
|AUTO POSTO DUQUE ...|      SAO PAULO|   SP|    SE|                       5.99|
|MASTER POSTO COME...|         SOBRAL|   CE|    NE|                       5.99|
|AUTO POSTO R&R AL...|        BARUERI|   SP|    SE|                       5.99|
|AUTO POSTO CATARI...|        VILHENA|  

# CNG Prices

In [ ]:
df_ethanol = df_fuels.filter(df_fuels["Product"] == "Ethanol")
df_ethanol.show()

+------+-----+-------------------+--------------------+-------+-----------+--------------+----------+
|Region|State|               City|         Gas Station|Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+-------------------+--------------------+-------+-----------+--------------+----------+
|    SE|   SP|           SOROCABA|COMPETRO COMERCIO...|Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|    CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|         RIO BRANCO|CODEPE COMBUSTIVE...|Ethanol|       5.29|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO CASTAN...|Ethanol|       5.49|          null|   EQUADOR|
|     N|   AC|         RIO BRANCO|AUTO POSTO CIDADE...|Ethanol|       5.29|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO DISTRI...|Ethanol|       5.49|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO SANTO ...|Ethanol|       5.32|       

### Ethanol Prices by Region

In [ ]:
df_ethanol.groupBy("Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price")) \
    .orderBy("Average Ethanol Sales Price", ascending=False) \
    .show()

+------+---------------------------+
|Region|Average Ethanol Sales Price|
+------+---------------------------+
|     N|                       5.11|
|    NE|                       4.91|
|     S|                        4.7|
|    SE|                        4.3|
|    CO|                       4.28|
+------+---------------------------+



### Ethanol Prices by State

In [ ]:
ethanol_by_state = df_ethanol.groupBy("State") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_state.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+-----+---------------------------+
|State|Average Ethanol Sales Price|
+-----+---------------------------+
|   AC|                       5.57|
|   AM|                       5.48|
|   AP|                       5.47|
|   RO|                       5.25|
|   CE|                       5.25|
|   RR|                       5.16|
|   MA|                       5.04|
|   AL|                       4.98|
|   RN|                       4.96|
|   RS|                       4.94|
+-----+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_state.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+-----+---------------------------+
|State|Average Ethanol Sales Price|
+-----+---------------------------+
|   MT|                       4.09|
|   SP|                       4.16|
|   MS|                        4.2|
|   GO|                       4.32|
|   MG|                       4.38|
|   PR|                       4.53|
|   ES|                       4.59|
|   PB|                       4.62|
|   RJ|                       4.69|
|   DF|                       4.71|
+-----+---------------------------+
only showing top 10 rows



### Ethanol Prices by City

In [ ]:
ethanol_by_city = df_ethanol.groupBy("City", "State", "Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_city.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+-----+------+---------------------------+
|                City|State|Region|Average Ethanol Sales Price|
+--------------------+-----+------+---------------------------+
|     CRUZEIRO DO SUL|   AC|     N|                       5.98|
|             VILHENA|   RO|     N|                       5.74|
|SANTANA DO LIVRAM...|   RS|     S|                       5.68|
|            ALTAMIRA|   PA|     N|                       5.61|
|           ITAPIPOCA|   CE|    NE|                        5.6|
|                CODO|   MA|    NE|                       5.57|
|              MANAUS|   AM|     N|                       5.49|
|              MACAPA|   AP|     N|                       5.47|
|             CRATEUS|   CE|    NE|                       5.47|
|         PORTO VELHO|   RO|     N|                       5.46|
+--------------------+-----+------+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_city.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+--------------------+-----+------+---------------------------+
|                City|State|Region|Average Ethanol Sales Price|
+--------------------+-----+------+---------------------------+
|                LINS|   SP|    SE|                       3.79|
|           ARACATUBA|   SP|    SE|                       3.79|
|SAO JOSE DO RIO P...|   SP|    SE|                       3.83|
|             BIRIGUI|   SP|    SE|                       3.85|
|            ANAPOLIS|   GO|    CO|                       3.87|
|            GOIATUBA|   GO|    CO|                       3.93|
|             MARILIA|   SP|    SE|                       3.94|
|          UBERLANDIA|   MG|    SE|                       3.94|
|              FRANCA|   SP|    SE|                       3.94|
|          ADAMANTINA|   SP|    SE|                       3.96|
+--------------------+-----+------+---------------------------+
only showing top 10 rows



### Ethanol Prices by Brand

In [ ]:
ethanol_by_brand = df_fuels.filter(col("Product") == "Ethanol") \
    .groupBy("Fuel Brand") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_brand.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+-----------------+---------------------------+
|       Fuel Brand|Average Ethanol Sales Price|
+-----------------+---------------------------+
|RZD DISTRIBUIDORA|                       5.49|
|          ATEM' S|                       5.47|
|          EQUADOR|                       5.35|
|              FAN|                       5.22|
|               SP|                       5.21|
|            SABBÁ|                       5.02|
|      RAIZEN MIME|                       4.99|
|      SANTA LUCIA|                       4.99|
|           DISLUB|                       4.96|
|         ON PETRO|                       4.93|
+-----------------+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_brand.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+----------------+---------------------------+
|      Fuel Brand|Average Ethanol Sales Price|
+----------------+---------------------------+
|       ROYAL FIC|                       3.79|
|          D`MAIS|                       4.03|
|SIM DISTRIBUIDOR|                       4.04|
|            WATT|                       4.16|
|           IDAZA|                       4.25|
|        CIAPETRO|                       4.27|
|          TAURUS|                       4.31|
|   TOTALENERGIES|                       4.37|
|      PETROSERRA|                        4.4|
|          BRANCA|                       4.41|
+----------------+---------------------------+
only showing top 10 rows



### Ethanol Prices by Gas Station

In [ ]:
df_fuels.show()

+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|Region|State|           City|         Gas Station|         Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|        Gasoline|       5.65|          null|    BRANCA|
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|         Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|        Gasoline|       8.09|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Premium Gasoline|       8.19|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|         Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|     RIO BRANCO|CODEPE COMBUSTIVE...|        Gasoline|       7.62|          null|     VIBRA|
|     N|   AC|     RIO BRANCO|CODEPE 

In [ ]:
ethanol_by_gas_station = df_fuels.filter(col("Product") == "Ethanol") \
    .groupBy("Gas Station", "City", "State", "Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_gas_station.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+---------------+-----+------+---------------------------+
|         Gas Station|           City|State|Region|Average Ethanol Sales Price|
+--------------------+---------------+-----+------+---------------------------+
|COMERCIAL ALTO SA...|  SERRA TALHADA|   PE|    NE|                       6.49|
|SADI DALLACORTE E...|        CHAPECO|   SC|     S|                       6.36|
|C M DERIVADOS DE ...|      FORTALEZA|   CE|    NE|                       6.09|
| S & G PETROLEO LTDA|CRUZEIRO DO SUL|   AC|     N|                       6.08|
|PG COMERCIO E TRA...|        VACARIA|   RS|     S|                       6.05|
| SKY AUTO POSTO LTDA|      SAO PAULO|   SP|    SE|                       5.99|
|AUTO POSTO DUQUE ...|      SAO PAULO|   SP|    SE|                       5.99|
|MASTER POSTO COME...|         SOBRAL|   CE|    NE|                       5.99|
|AUTO POSTO R&R AL...|        BARUERI|   SP|    SE|                       5.99|
|AUTO POSTO CATARI...|        VILHENA|  

In [ ]:
ethanol_by_gas_station.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+---------------+-----+------+---------------------------+
|         Gas Station|           City|State|Region|Average Ethanol Sales Price|
+--------------------+---------------+-----+------+---------------------------+
|COMERCIAL ALTO SA...|  SERRA TALHADA|   PE|    NE|                       6.49|
|SADI DALLACORTE E...|        CHAPECO|   SC|     S|                       6.36|
|C M DERIVADOS DE ...|      FORTALEZA|   CE|    NE|                       6.09|
| S & G PETROLEO LTDA|CRUZEIRO DO SUL|   AC|     N|                       6.08|
|PG COMERCIO E TRA...|        VACARIA|   RS|     S|                       6.05|
| SKY AUTO POSTO LTDA|      SAO PAULO|   SP|    SE|                       5.99|
|AUTO POSTO DUQUE ...|      SAO PAULO|   SP|    SE|                       5.99|
|MASTER POSTO COME...|         SOBRAL|   CE|    NE|                       5.99|
|AUTO POSTO R&R AL...|        BARUERI|   SP|    SE|                       5.99|
|AUTO POSTO CATARI...|        VILHENA|  

# LPG Prices

In [ ]:
df_ethanol = df_fuels.filter(df_fuels["Product"] == "Ethanol")
df_ethanol.show()

+------+-----+-------------------+--------------------+-------+-----------+--------------+----------+
|Region|State|               City|         Gas Station|Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+-------------------+--------------------+-------+-----------+--------------+----------+
|    SE|   SP|           SOROCABA|COMPETRO COMERCIO...|Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|    CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|         RIO BRANCO|CODEPE COMBUSTIVE...|Ethanol|       5.29|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO CASTAN...|Ethanol|       5.49|          null|   EQUADOR|
|     N|   AC|         RIO BRANCO|AUTO POSTO CIDADE...|Ethanol|       5.29|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO DISTRI...|Ethanol|       5.49|          null|     VIBRA|
|     N|   AC|         RIO BRANCO|AUTO POSTO SANTO ...|Ethanol|       5.32|       

### Ethanol Prices by Region

In [ ]:
df_ethanol.groupBy("Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price")) \
    .orderBy("Average Ethanol Sales Price", ascending=False) \
    .show()

+------+---------------------------+
|Region|Average Ethanol Sales Price|
+------+---------------------------+
|     N|                       5.11|
|    NE|                       4.91|
|     S|                        4.7|
|    SE|                        4.3|
|    CO|                       4.28|
+------+---------------------------+



### Ethanol Prices by State

In [ ]:
ethanol_by_state = df_ethanol.groupBy("State") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_state.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+-----+---------------------------+
|State|Average Ethanol Sales Price|
+-----+---------------------------+
|   AC|                       5.57|
|   AM|                       5.48|
|   AP|                       5.47|
|   RO|                       5.25|
|   CE|                       5.25|
|   RR|                       5.16|
|   MA|                       5.04|
|   AL|                       4.98|
|   RN|                       4.96|
|   RS|                       4.94|
+-----+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_state.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+-----+---------------------------+
|State|Average Ethanol Sales Price|
+-----+---------------------------+
|   MT|                       4.09|
|   SP|                       4.16|
|   MS|                        4.2|
|   GO|                       4.32|
|   MG|                       4.38|
|   PR|                       4.53|
|   ES|                       4.59|
|   PB|                       4.62|
|   RJ|                       4.69|
|   DF|                       4.71|
+-----+---------------------------+
only showing top 10 rows



### Ethanol Prices by City

In [ ]:
ethanol_by_city = df_ethanol.groupBy("City", "State", "Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_city.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+-----+------+---------------------------+
|                City|State|Region|Average Ethanol Sales Price|
+--------------------+-----+------+---------------------------+
|     CRUZEIRO DO SUL|   AC|     N|                       5.98|
|             VILHENA|   RO|     N|                       5.74|
|SANTANA DO LIVRAM...|   RS|     S|                       5.68|
|            ALTAMIRA|   PA|     N|                       5.61|
|           ITAPIPOCA|   CE|    NE|                        5.6|
|                CODO|   MA|    NE|                       5.57|
|              MANAUS|   AM|     N|                       5.49|
|              MACAPA|   AP|     N|                       5.47|
|             CRATEUS|   CE|    NE|                       5.47|
|         PORTO VELHO|   RO|     N|                       5.46|
+--------------------+-----+------+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_city.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+--------------------+-----+------+---------------------------+
|                City|State|Region|Average Ethanol Sales Price|
+--------------------+-----+------+---------------------------+
|                LINS|   SP|    SE|                       3.79|
|           ARACATUBA|   SP|    SE|                       3.79|
|SAO JOSE DO RIO P...|   SP|    SE|                       3.83|
|             BIRIGUI|   SP|    SE|                       3.85|
|            ANAPOLIS|   GO|    CO|                       3.87|
|            GOIATUBA|   GO|    CO|                       3.93|
|             MARILIA|   SP|    SE|                       3.94|
|          UBERLANDIA|   MG|    SE|                       3.94|
|              FRANCA|   SP|    SE|                       3.94|
|          ADAMANTINA|   SP|    SE|                       3.96|
+--------------------+-----+------+---------------------------+
only showing top 10 rows



### Ethanol Prices by Brand

In [ ]:
ethanol_by_brand = df_fuels.filter(col("Product") == "Ethanol") \
    .groupBy("Fuel Brand") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_brand.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+-----------------+---------------------------+
|       Fuel Brand|Average Ethanol Sales Price|
+-----------------+---------------------------+
|RZD DISTRIBUIDORA|                       5.49|
|          ATEM' S|                       5.47|
|          EQUADOR|                       5.35|
|              FAN|                       5.22|
|               SP|                       5.21|
|            SABBÁ|                       5.02|
|      RAIZEN MIME|                       4.99|
|      SANTA LUCIA|                       4.99|
|           DISLUB|                       4.96|
|         ON PETRO|                       4.93|
+-----------------+---------------------------+
only showing top 10 rows



In [ ]:
ethanol_by_brand.orderBy("Average Ethanol Sales Price", ascending=True).show(10)

+----------------+---------------------------+
|      Fuel Brand|Average Ethanol Sales Price|
+----------------+---------------------------+
|       ROYAL FIC|                       3.79|
|          D`MAIS|                       4.03|
|SIM DISTRIBUIDOR|                       4.04|
|            WATT|                       4.16|
|           IDAZA|                       4.25|
|        CIAPETRO|                       4.27|
|          TAURUS|                       4.31|
|   TOTALENERGIES|                       4.37|
|      PETROSERRA|                        4.4|
|          BRANCA|                       4.41|
+----------------+---------------------------+
only showing top 10 rows



### Ethanol Prices by Gas Station

In [ ]:
df_fuels.show()

+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|Region|State|           City|         Gas Station|         Product|Sales Price|Purchase Price|Fuel Brand|
+------+-----+---------------+--------------------+----------------+-----------+--------------+----------+
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|        Gasoline|       5.65|          null|    BRANCA|
|    SE|   SP|       SOROCABA|COMPETRO COMERCIO...|         Ethanol|       3.78|          null|    BRANCA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|        Gasoline|       8.09|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|Premium Gasoline|       8.19|          null|  IPIRANGA|
|     N|   AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...|         Ethanol|       5.95|          null|  IPIRANGA|
|     N|   AC|     RIO BRANCO|CODEPE COMBUSTIVE...|        Gasoline|       7.62|          null|     VIBRA|
|     N|   AC|     RIO BRANCO|CODEPE 

In [ ]:
ethanol_by_gas_station = df_fuels.filter(col("Product") == "Ethanol") \
    .groupBy("Gas Station", "City", "State", "Region") \
    .agg(round(avg("Sales Price"), 2).alias("Average Ethanol Sales Price"))

In [ ]:
ethanol_by_gas_station.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+---------------+-----+------+---------------------------+
|         Gas Station|           City|State|Region|Average Ethanol Sales Price|
+--------------------+---------------+-----+------+---------------------------+
|COMERCIAL ALTO SA...|  SERRA TALHADA|   PE|    NE|                       6.49|
|SADI DALLACORTE E...|        CHAPECO|   SC|     S|                       6.36|
|C M DERIVADOS DE ...|      FORTALEZA|   CE|    NE|                       6.09|
| S & G PETROLEO LTDA|CRUZEIRO DO SUL|   AC|     N|                       6.08|
|PG COMERCIO E TRA...|        VACARIA|   RS|     S|                       6.05|
| SKY AUTO POSTO LTDA|      SAO PAULO|   SP|    SE|                       5.99|
|AUTO POSTO DUQUE ...|      SAO PAULO|   SP|    SE|                       5.99|
|MASTER POSTO COME...|         SOBRAL|   CE|    NE|                       5.99|
|AUTO POSTO R&R AL...|        BARUERI|   SP|    SE|                       5.99|
|AUTO POSTO CATARI...|        VILHENA|  

In [ ]:
ethanol_by_gas_station.orderBy("Average Ethanol Sales Price", ascending=False).show(10)

+--------------------+---------------+-----+------+---------------------------+
|         Gas Station|           City|State|Region|Average Ethanol Sales Price|
+--------------------+---------------+-----+------+---------------------------+
|COMERCIAL ALTO SA...|  SERRA TALHADA|   PE|    NE|                       6.49|
|SADI DALLACORTE E...|        CHAPECO|   SC|     S|                       6.36|
|C M DERIVADOS DE ...|      FORTALEZA|   CE|    NE|                       6.09|
| S & G PETROLEO LTDA|CRUZEIRO DO SUL|   AC|     N|                       6.08|
|PG COMERCIO E TRA...|        VACARIA|   RS|     S|                       6.05|
| SKY AUTO POSTO LTDA|      SAO PAULO|   SP|    SE|                       5.99|
|AUTO POSTO DUQUE ...|      SAO PAULO|   SP|    SE|                       5.99|
|MASTER POSTO COME...|         SOBRAL|   CE|    NE|                       5.99|
|AUTO POSTO R&R AL...|        BARUERI|   SP|    SE|                       5.99|
|AUTO POSTO CATARI...|        VILHENA|  